### following https://pytorch.org/hub/facebookresearch_pytorchvideo_resnet/

In [38]:
import pytorch_saliency as ps
import torch
import torchvision.transforms as T

from pytorchvideo.data.encoded_video import EncodedVideo
import pytorchvideo.transforms as TV

In [ ]:
#load pre-trained 3D resnet trained on Kinetics 400
resnet3d = torch.hub.load('facebookresearch/pytorchvideo', 'slow_r50', pretrained=True)
resnet3d.eval()

In [7]:
import urllib
import json

# downlaod labels
json_url = "https://dl.fbaipublicfiles.com/pyslowfast/dataset/class_names/kinetics_classnames.json"
json_filename = "kinetics_classnames.json"
try: urllib.URLopener().retrieve(json_url, json_filename)
except: urllib.request.urlretrieve(json_url, json_filename)

In [8]:
# create dict with int label -> text label
with open(json_filename, "r") as f:
    class_names = json.load(f)
classes = {i : label.replace('"', "") for label, i in class_names.items()}


In [27]:
from torchvision.transforms._transforms_video import CenterCropVideo

/Users/subrasal/saj_env/lib/python3.9/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in 0.14. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/Users/subrasal/saj_env/lib/python3.9/site-packages/torchvision/transforms/_transforms_video.py:25: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in 0.14. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [28]:
# slow_R50 specific pre-processing
side_size = 256
mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
crop_size = 256
num_frames = 8
sampling_rate = 8
frames_per_second = 30

transform =  TV.ApplyTransformToKey(
    key="video",
    transform=T.Compose(
        [
            TV.UniformTemporalSubsample(num_frames),
            T.Lambda(lambda x: x/255.0),
            T._transforms_video.NormalizeVideo(mean, std),
            TV.ShortSideScale(
                size=side_size
            ),
            T._transforms_video.CenterCropVideo(crop_size=(crop_size, crop_size))
        ]
    ),
)

# The duration of the input clip is also specific to the model.
clip_duration = (num_frames * sampling_rate)/frames_per_second

In [30]:
# download video
url_link = "https://dl.fbaipublicfiles.com/pytorchvideo/projects/archery.mp4"
video_path = 'archery.mp4'
try: urllib.URLopener().retrieve(url_link, video_path)
except: urllib.request.urlretrieve(url_link, video_path)

In [39]:
video = EncodedVideo.from_path(video_path)

In [42]:
clip = video.get_clip(start_sec=0, end_sec=clip_duration)
tensor = clip["video"]

In [45]:
tens = clip["video"]

In [64]:
ps.utils.plot(tens.unsqueeze(0).int(), interactive=True)

interactive(children=(IntSlider(value=32, description='Slice:', max=63), Output()), _dom_classes=('widget-inte…